In [2]:
import gensim.models.keyedvectors as word2vec
from gensim.scripts.glove2word2vec import glove2word2vec
import gensim.models.fasttext as ft
import time
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import re
from sklearn.linear_model import LogisticRegression

In [3]:
CLEANED_NAMES_FILE = "../Data/category_cleaned_names.csv"

ft_wiki_model = word2vec.KeyedVectors.load_word2vec_format('../Data/wiki-news-300d-1M-subword.vec')
ft_wiki_vocab = [key.lower() for key in ft_wiki_model.vocab.keys()]


ft_wiki_vocab = [key.lower() for key in ft_wiki_model.vocab.keys()]


w2v_model = word2vec.KeyedVectors.load_word2vec_format('../Data/GoogleNews-vectors-negative300.bin', binary=True)
w2v_vocab = [key.lower() for key in w2v_model.vocab.keys()]

# glove_model_load = glove2word2vec("../Data/glove.840B.300d.txt", "../Data/gensim_glove_vectors.txt")
glove_model = word2vec.KeyedVectors.load_word2vec_format("../Data/gensim_glove_vectors.txt")


glove_vocab = [key.lower() for key in glove_model.vocab.keys()]

In [4]:
# REPEAT CODE WITH FASTTEXT
df_models = pd.read_csv('../Data/all_models_alignment_11_06.csv')
df_models = df_models[df_models.columns[2:]]
pearsonr(df_models['Glove_alignment'], df_models['Word2Vec_alignment'])
pearsonr(df_models['Wikipedia_alignment'], df_models['Word2Vec_alignment'])
pearsonr(df_models['Glove_alignment'], df_models['Wikipedia_alignment'])
df_models_trim =df_models.replace(0, np.nan).dropna()
print(pearsonr(df_models_trim['Glove_alignment'], df_models_trim['Word2Vec_alignment']))
print(pearsonr(df_models_trim['Wikipedia_alignment'], df_models_trim['Word2Vec_alignment']))
print(pearsonr(df_models_trim['Glove_alignment'], df_models_trim['Wikipedia_alignment']))
print(pearsonr(df_models_trim['Glove_resid_alignment'], df_models_trim['Word2Vec_resid_alignment']))
print(pearsonr(df_models_trim['Wikipedia_resid_alignment'], df_models_trim['Word2Vec_resid_alignment']))
print(pearsonr(df_models_trim['Glove_resid_alignment'], df_models_trim['Wikipedia_resid_alignment']))

(0.6712605153522405, 3.377764922892809e-166)
(0.4112802530312509, 9.867871380647285e-53)
(0.44779638845765096, 2.622405221877182e-63)
(0.6349455464765613, 1.6926896111528896e-143)
(0.5172240051202465, 2.4567665029618766e-87)
(0.4339555372724522, 3.807155025282788e-59)


In [5]:
def get_ss_values(valid_brands):
    ratio_df_human = pd.read_csv('../Data/human_names_coded.csv')
    ratio_df_human = ratio_df_human.drop(["dataID"], axis=1)
    ratio_df_human = ratio_df_human.drop(["Name"], axis=1)
    ratio_x = ratio_df_human.drop(['Gender', 'is_unique'], axis=1).as_matrix()[:, 1:]
    ratio_y = ratio_df_human[['Gender']].as_matrix().ravel()
    ratio_model_LR = LogisticRegression()
    ratio_model_LR.fit(ratio_x, ratio_y)
    ratio_df_brands = pd.read_csv('../Data/brand_names_coded_no_duplicates.csv')
    ratio_maps = {}
    for i in range(len(ratio_df_brands.index)):
        feature_vec = ratio_df_brands.iloc[i][4:].as_matrix().ravel().reshape((1, -1))
        x = ratio_model_LR.predict_proba(feature_vec)
#         print(ratio_df_brands.iloc[i][2], x)
        ratio_maps[ratio_df_brands.iloc[i][2]] = x[0][1]
    valid_alignments = []
    for brand in valid_brands:
        valid_alignments.append(ratio_maps[brand])
    return valid_alignments
        

In [6]:
df_models_trim['ss_alignment'] = get_ss_values(df_models_trim['brand'])

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

...

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .value

In [7]:
web_df = pd.read_csv('../Data/filtered_web_brands.csv', header=0)[['Name', 'Category', 'Overall Prediction', 'Gender in Photos Ratio (M:F)']]
web_df['mal'] = web_df['Gender in Photos Ratio (M:F)'].apply(lambda x: int(x.split(":")[0]))
web_df['fal'] = web_df['Gender in Photos Ratio (M:F)'].apply(lambda x: int(x.split(":")[1]))
# ratio_web_df['alignment'] = ratio_web_df['fal'] / ratio_web_df['mal']
web_df['alignment'] = (web_df['fal'] - web_df['mal']) / web_df[['fal', 'mal']].max(axis=1)
web_df['total'] = web_df['fal'] + web_df['mal']
web_df = web_df[web_df['total'] > 2]
sort_web_df = web_df.sort_values(by='Name')
sort_models_df = df_models_trim[df_models_trim['brand'].isin(sort_web_df['Name'].tolist())].sort_values(by='brand')
sort_web_df = sort_web_df[sort_web_df['Name'].isin(sort_models_df['brand'].tolist())]

In [9]:
base_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Word2Vec_alignment'])
bert_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Wikipedia_alignment'])
corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Glove_alignment'])
category_corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Word2Vec_resid_alignment'])
residual_bert_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Wikipedia_resid_alignment'])
residual_corp_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['Glove_resid_alignment'])
ss_correlation = pearsonr(sort_web_df['alignment'], sort_models_df['ss_alignment'])

In [10]:
print(f"Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: {np.round(base_correlation[0], 4)}, p-value: {np.round(base_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Raw Fasttext (Wikipedia) Embeddings: {np.round(bert_correlation[0], 4)}, p-value: {np.round(bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Raw Glove Embeddings: {np.round(corp_correlation[0], 4)}, p-value: {np.round(corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Word2Vec Embeddings: {np.round(category_corp_correlation[0], 4)}, p-value: {np.round(category_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Fasttest (Wikipedia) Embeddings: {np.round(residual_bert_correlation[0], 4)}, p-value: {np.round(residual_bert_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Residual Glove Embeddings: {np.round(residual_corp_correlation[0], 4)}, p-value: {np.round(residual_corp_correlation[1], 4)}")
print(f"Correlation of Image Gender Ratio and Sound Symbolism: {np.round(ss_correlation[0], 4)}, p-value: {np.round(ss_correlation[1], 4)}")


Correlation of Image Gender Ratio and Raw Word2Vec Embeddings: 0.4101, p-value: 0.0
Correlation of Image Gender Ratio and Raw Fasttext (Wikipedia) Embeddings: 0.2589, p-value: 0.0
Correlation of Image Gender Ratio and Raw Glove Embeddings: 0.4233, p-value: 0.0
Correlation of Image Gender Ratio and Residual Word2Vec Embeddings: 0.0739, p-value: 0.2191
Correlation of Image Gender Ratio and Residual Fasttest (Wikipedia) Embeddings: 0.0749, p-value: 0.2132
Correlation of Image Gender Ratio and Residual Glove Embeddings: -0.0374, p-value: 0.5349
Correlation of Image Gender Ratio and Sound Symbolism: 0.1715, p-value: 0.0041


In [11]:
def get_brand_categories(brands, concise=False):
    df_brands_category = pd.read_csv('../Data/brand_names_coded.csv')
    df_brands_no_dup = df_brands_category.drop_duplicates(subset='BRAND')
    if concise:
        cleaned_name_map = pd.read_csv(CLEANED_NAMES_FILE).set_index('Original').to_dict()['Concise']
    else:
        cleaned_name_map = pd.read_csv(CLEANED_NAMES_FILE).set_index('Original').to_dict()['Corrected']
    valid_df = df_brands_no_dup[df_brands_no_dup['BRAND'].isin(brands)]
    brand_category_mapping = valid_df.set_index('BRAND').to_dict()['CATEGORY']
    brand_category_cleaned = {brand: preprocess(cleaned_name_map[category]) for brand, category in brand_category_mapping.items()}
#     brand_category_mapping = {brand: preprocess(category) for brand, category in brand_category_cleaned.items()}
    return brand_category_cleaned, list(set(brand_category_cleaned.values()))

In [14]:
def preprocess(category):
    category = category[5:]
    category = re.sub(",|'s|& |/", ' ', category)
    category = re.sub("  ", " ", category )
    return category.lower()

In [15]:
brands = web_df['Name'].tolist()

brand_category_mapping, categories = get_brand_categories(brands)
categories = list(categories)

In [20]:
sort_models_df['Category'] = sort_models_df['brand'].apply(lambda x: brand_category_mapping[x])


,brand,alignment,Word2Vec_alignment,Word2Vec_resid_alignment,Wikipedia_alignment,Wikipedia_resid_alignment,Glove_alignment,Glove_resid_alignment,ss_alignment,Category
1320,abbotts,0.145014,0.145014,0.089758,-0.158946,-0.012121,0.057109,0.004442,0.053954,ice cream frozen novelties sherbet
1325,abuelos,0.145999,0.145999,0.023558,-0.070452,0.039857,0.001983,-0.032581,0.063295,casual dining restaurants
55,acdelco,-0.072280,-0.072280,-0.082613,-0.171089,-0.061498,-0.054666,-0.046685,0.466647,household batteries
366,acme,-0.001673,-0.001673,0.083177,-0.122985,-0.002659,-0.124430,0.019829,0.667916,fishing tackle equipment
706,advil,0.038978,0.038978,-0.037600,-0.046704,0.078058,0.044699,-0.035485,0.238638,pain remedies
1379,akris,0.169658,0.169658,-0.084352,-0.082522,-0.092325,0.015157,-0.085337,0.260092,handbags
393,aleve,0.001270,0.001270,-0.062260,-0.115402,0.023315,0.046550,-0.030200,0.727760,pain remedies
421,align,0.004917,0.004917,-0.072987,-0.131412,-0.113290,-0.018403,-0.082460,0.323157,nutritional supplements
334,alitalia,-0.007451,-0.007451,-0.020659,-0.098762,0.007900,-0.006055,-0.013077,0.989046,travel services
427,altra,0.006131,0.006131,0.014969,-0.124524,0.000743,-0.002678,-0.029014,0.973987,sneakers


In [25]:
count_df = sort_models_df.groupby(by='Category').count()
frequent_categories = count_df[count_df['brand'] > 5].index.tolist()
shortened_df = sort_models_df[sort_models_df['Category'].isin(frequent_categories)]
prefixes = ['Word2Vec', 'Wikipedia', 'Glove']
for resid in ['', '_resid']:
    for pref in prefixes:
        shortened_df[f'gender_{pref}{resid}'] = shortened_df[f'{pref}{resid}_alignment'].apply(lambda x: 1 if x >=0 else 0)
category_counts = shortened_df.groupby(by='Category').sum()
for resid in ['', '_resid']:
    for pref in prefixes:
        category_counts[f'female_{pref}{resid}'] = category_counts[f'gender_{pref}{resid}'] / count_df['brand']
category_counts

c:\users\jai\desktop\bgp-wordembedding\bgp_venv\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,alignment,Word2Vec_alignment,Word2Vec_resid_alignment,Wikipedia_alignment,Wikipedia_resid_alignment,Glove_alignment,Glove_resid_alignment,ss_alignment,gender_Word2Vec,gender_Wikipedia,gender_Glove,gender_Word2Vec_resid,gender_Wikipedia_resid,gender_Glove_resid,female_Word2Vec,female_Wikipedia,female_Glove,female_Word2Vec_resid,female_Wikipedia_resid,female_Glove_resid
Category,,,,,,,,,,,,,,,,,,,,
beer ale,-0.110880,-0.110880,0.486948,-2.336027,1.173802,-0.617908,0.221410,4.670081,6,0,5,11,15,9,0.400000,0.000000,0.333333,0.733333,1.000000,0.600000
casual dining restaurants,0.587996,0.587996,0.158587,-0.847402,-0.113977,0.084935,-0.194689,2.499199,6,0,5,5,2,1,0.857143,0.000000,0.714286,0.714286,0.285714,0.142857
credit cards,0.214957,0.214957,0.264134,-0.746714,0.357104,0.071669,0.198987,1.942097,5,0,3,5,6,5,0.833333,0.000000,0.500000,0.833333,1.000000,0.833333
cruise ship travel,0.375975,0.375975,0.134364,-0.335956,0.285702,0.485977,0.103165,3.455784,5,1,6,3,6,3,0.833333,0.166667,1.000000,0.500000,1.000000,0.500000
fishing tackle equipment,0.439663,0.439663,1.191535,-1.447957,0.331690,-0.358896,0.723995,3.419316,8,0,2,10,5,10,0.800000,0.000000,0.200000,1.000000,0.500000,1.000000
handbags,2.446111,2.446111,-1.627545,-1.681143,-1.676150,1.315400,-1.220088,10.203693,17,1,16,2,1,2,0.894737,0.052632,0.842105,0.105263,0.052632,0.105263
mattresses box springs,1.135901,1.135901,0.953495,-1.666920,0.254788,0.484553,0.280269,6.378544,11,0,10,11,10,9,0.846154,0.000000,0.769231,0.846154,0.769231,0.692308
nutritional supplements,1.098172,1.098172,-1.236218,-2.651238,-2.013419,0.019445,-1.478369,9.969094,18,1,11,3,1,3,0.782609,0.043478,0.478261,0.130435,0.043478,0.130435
security fire safety systems,0.264479,0.264479,0.481120,-2.688935,-0.342150,-0.569829,0.461078,6.699683,12,0,5,14,7,15,0.666667,0.000000,0.277778,0.777778,0.388889,0.833333


In [31]:
for resid in ['', '_resid']:
    for pref in prefixes:
        
        old_order = category_counts.sort_values(by=f'female_{pref}{resid}')  
        cat_ind = old_order.index
        indices = [i for i in range(len(cat_ind))]
        sub_m = (1 - old_order[f'female_{pref}{resid}']).tolist()
        sub_f = old_order[f'female_{pref}{resid}'].tolist()
        sub_c = old_order.index
        barWidth = 0.5
        plt.bar(indices, sub_m, color='#0b6304', width=barWidth, label="Male")
        plt.bar(indices, sub_f, bottom=sub_m, color='#f78725', width=barWidth, label="Female")
        plt.xticks(indices, sub_c, rotation='vertical', fontsize=8)
        plt.xlabel("Categories")
        plt.ylabel("Proportion (%)")
        plt.title(f"Gender Distribution across {pref}{resid} Categories")
        plt.legend()
        plt.axhline(y=0.5)
        plt.rcParams["figure.figsize"] = [10, 5]
        plt.gcf().subplots_adjust(bottom=0.50)
#         plt.show()
        plt.savefig(f'../Plots/category_distribution_{pref}{resid}_07_15.png', dpi=150)
        plt.clf()
# plt.clf()


<Figure size 720x360 with 0 Axes>